# Dealing with dates and years

In [ ]:
import pandas as pd
from datetime import datetime
import re

df = pd.read_excel("GSAF5.xls")

## Normalize dates

- `Reported` was removed from the date column
- `Sept` is not an accepted `datetime` so it was replaced with `Sep`
- There was a typo in one record `Nox` instead of `Nov`

In [ ]:
def normalize_date(date):
    if isinstance(date, datetime): return date
    date = str(date).strip().lower()
    date = date.replace("reported", "")
    date = date.replace("september", "sep")
    date = date.replace("sept", "sep")
    date = date.replace("nox", "nov")
    return date

## Format dates

### Defining regular expressions

- **Short dates:** 30-Oct-2024 or similar
- **Long dates:** 30-October-2024 or similar
- **Years**: 4 consecutive numbers

### Running regular expressions
- If the date is already a Python `datetime` object, return the date as it is
- First match long dates, short dates then years
- Parse these dates into a Python `datetime` object
- If there are any errors, return the date as it is

In [ ]:
any = ".*?"
numbers = "[0-9]"
letters = "[a-zA-Z]"
separators = "[-\\s]"

short_date_regex = f"^{any}({numbers}{1,2}){separators}+({letters}{3,4}){separators}+({numbers}{4}){any}$"
long_date_regex = f"^{any}({numbers}{1,2}){separators}+({letters}{4,10}){separators}+({numbers}{4}){any}$"

year_regex = "^[0-9]{4}$"

def match_date(regex, date):
    match = re.search(regex, date)
    if match:
        day = long_match.group(1).zfill(2)
        month = long_match.group(2)
        year = long_match.group(3)
        return datetime.strptime(f"{day}/{month}/{year}", "%d/%B/%Y")
    return None

def format_date(date):
    if isinstance(date, datetime): return date

    try:
        long_date = match_date(long_date_regex, date)
        if long_date: return long_date

        short_date = match_date(short_date_regex, date)
        if short_date: return short_date

        year_match = re.search(year_regex, date)
        if year_match: return None # datetime.strptime(year_match.group(0), "%Y")
    except ValueError: return date

## Normalize years

- For years greater than or equal to 1000, return them as they are
- For dates between 100 and 999 (e.g. 950), add `1` to the beginning (e.g. 1950)
- For dates between 25 and 99 (e.g. 93), add `19` to the beginning (e.g. 1993)
- For dates between 10 and 24 (e.g. 24), add `20` to the beginning (e.g. 2024)
- For dates between 0 and 9 (e.g. 4), add `200` to the beginning (e.g. 2004)

In [ ]:
def normalize_year(year):
    if year >= 1000: return year
    if year >= 100: return float(f"1{year}")
    if year >= 25: return float(f"19{year}")
    if year >= 10: return float(f"20{year}")
    if year >= 0: return float(f"200{year}")
    return year

## Infer years from dates

If a year is missing and the date record is present, use the year part from the date column

In [ ]:
def infer_year(row):
    if pd.isnull(row.Year): row.Year = row.Date.year
    return row

## Apply functions to DataFrame

1. Normalize date
2. Format date
3. Drop empty dates otherwise
4. Normalize year
5. Infer year

In [ ]:
df_copy = df.copy()

df_copy.Date = df_copy.Date.apply(normalize_date)
df_copy.Date = df_copy.Date.apply(format_date)
df_copy = df_copy.dropna(subset="Date")

df_copy.Year = df_copy.Year.apply(normalize_year)
df_copy[["Date", "Year"]] = df_copy[["Date", "Year"]].apply(infer_year, axis=1)

In [ ]:
import re

sharks_df = df_copy.copy()

sharks_df['Country'] = sharks_df['Country'].str.split(' / ')

#Explode the Country lists into separate rows
sharks_df = sharks_df.explode('Country')
sharks_df["Country"].unique()
#sharks_df["Country"].nunique()

#replace bad entries with a single country but too repetitive doesn't show true data cleaning
sharks_df['Country'] = sharks_df['Country'].replace({'CEYLON (SRI LANKA)': 'Sri Lanka'})
sharks_df['Country'] = sharks_df['Country'].replace({'ST HELENA, British overseas territory': 'Saint Helena'})



sharks_df["Country"] = sharks_df["Country"].str.lower()
sharks_df = sharks_df.dropna(subset=['Country'])

#sharks_df["Country"] = sharks_df["Country"].replace({"?": ""})
#Call method strip to strip off question marks at the end
sharks_df['Country'] = sharks_df['Country'].str.strip('?')

#call is in method to remove oceans+seas+continents

# Wordslist to remove
words_to_remove = ['Asia', 'Africa' , 'ocean' , 'sea']
mask = ~sharks_df['Country'].isin(words_to_remove)
#df_cleaned = df[mask]

# Display cleaned DataFrame
#print("\nCleaned DataFrame:")
#print(df_cleaned)

def month_grouping(date):
    return date.strftime("%B")

#print(month_grouping(datetime(2024, 1, 1, 0, 0, 0)))
#print(month_grouping(datetime(2024, 2, 1, 0, 0, 0)))
#print(month_grouping(datetime(2024, 3, 1, 0, 0, 0)))

sharks_df["Month"] = sharks_df["Date"].apply(month_grouping)

sharks_df["Time"].unique()

time_df = sharks_df.copy()

any = ".*?"
numbers = "[0-9]"
letters = "[a-zA-Z]"
separators = "[hr\\s\\.]"

#long_time_regex = f"({numbers}{1,2}){separators}*({numbers}{2})"
long_time_regex = "([0-9]{1,2})[hr\\s\\.]*([0-9]{2})"
#short_time_regex = f"({numbers}{1,2})"
short_time_regex = "([0-9]{1,2})[hr\\s\\.]*"

def match_time(regex, time):
    match = re.search(regex, time)
    if match: return match
    return None
    #if match:
    #    hours = match.group(1).zfill(2)
    #    minutes = match.group(2)
    #    return f"{hours}:{minutes}"
    #return None
    
def sanitize_time(time):

    if time in ["Not advised", "Not stated", "?"]: return None
    if not isinstance(time, str): time = str(time)
    
    try:
        long_time = match_time(long_time_regex, time)
        if long_time:
            hours = long_time.group(1).zfill(2)
            minutes = long_time.group(2)
            return f"{hours}:{minutes}"

        short_time = match_time(short_time_regex, time)
        if short_time:
            hours = short_time.group(1).zfill(2)
            return f"{hours}:00"

        return None
    except ValueError: return time

time_df["Time"].unique()
time_df["Time"].apply(sanitize_time).unique()


# add time to Date column

#sharks_df.pivot(index=["Date", "Country", "State"], columns="Type", values="Injury")
# sharks_df.pivot(index["State", "Month"]).count().unstack()
#grouped = sharks_df.groupby(["State", "Country"])["State"].unique().unstack()

#grouped